In [1]:
import grequests
import requests
from datetime import datetime
from IPython.display import clear_output
import pandas as pd
import os

C:\Python310\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


# Собираем список id всех объектов

In [2]:
start = datetime.now()

offset_ = 1
limit_ = 100
a_urls = []

for i in range(110): # 110 - волшебное число, да не хорошо, но если делать нормально через вычитывание всего, то очень долго проверять, а так методом тыка стало известно, что там меньше 11к записей
    a_urls.append(f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_ * i * limit_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0')
    
async_list = []

rs = [grequests.get(u) for u in a_urls]
objids = []
for res in grequests.imap(rs, size=16):
    if not res.ok:
        continue
        
    objects_data = res.json()
    objects_list = objects_data.get('data').get('list')
    if not objects_list:
        continue
    
    clear_output()
    objids.extend(x.get('objId') for x in objects_list)
    print(len(objids))

print(len(objids))
print(datetime.now() - start)

10592
10592
0:00:08.977521


# Собираем все возможные объекты по полученным прежде id

In [3]:
full_objects = []


a_fullObj_urls = []
for objId in objids:
    a_fullObj_urls.append(f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{objId}')

rs_objects = [grequests.get(u) for u in a_fullObj_urls]

for res in grequests.imap(rs_objects, size=16):
    if not res.ok:
        continue
        
    object_data = res.json()
    full_obj = object_data.get('data')

    if not full_obj:
        continue
    
    clear_output()
    full_objects.append(full_obj)
    print()
    print(f'{len(full_objects)}/{len(objids)}')

print(len(full_objects))



10426/10592
10426


In [4]:
df = pd.json_normalize(full_objects)
out_filename = "./output/output.xlsx"
os.makedirs(os.path.dirname(out_filename), exist_ok=True)
df.to_excel(out_filename)  
df

,id,pdId,region,address,nameObj,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,metro.line,metro.color,metro.time,metro.isWalk,metro.colors,complexShortNm,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,19050,44823,54,"г Новосибирск, район Октябрьский, Улица Шевчен...",Жилой комплекс Академия,25.0,25.0,207,2026-06-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34698,249628,50,"п Зеленый, район Стрешнево, д. 1","ЖК ""Стрешнево""",17.0,17.0,256,2025-06-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38847,414321,58,"с Засечное, ул Новая, д. 17","ЖК ""Семейный""",11.0,11.0,243,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,43932,534097,64,"г Саратов, д. 7",NaN,12.0,12.0,150,2023-12-31,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,39811,435277,64,"г Саратов, 3-я жилая группа микрорайона № 9 жи...",NaN,11.0,11.0,238,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10421,28071,99284,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...","Жилой комплекс""Томилинский бульвар""",26.0,26.0,288,2024-06-30,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10422,44447,475093,76,"г Ярославль, Фрунзенский район, территория в р...",NaN,6.0,6.0,127,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10423,28074,96537,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...","Жилой комплекс""Томилинский бульвар""",2.0,26.0,600,2023-06-30,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10424,28072,96379,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...","Жилой комплекс""Томилинский бульвар""",2.0,26.0,312,2024-06-30,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


C:\Python310\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [5]:
def download_photo(): # сделал скачивание картинки с конкретного объекта, можно заменить на цикл и всё будет работать, но с 11к объектов качать картинки такое себе
    id = full_objects[0].get('id')
    photo_obj = full_objects[0].get('photoRenderDTO')
    
    if not photo_obj: # значит картинки нет
        return
    
    link = photo_obj[0].get('objRenderPhotoUrl') # так же качаю первую картинку, тоже можно заменить на цикл, если сильно захотеть
    resp = requests.get(link).content
    filename = f'./output/images/{id}.jpg' 
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'wb') as f:
        f.write(resp)

download_photo()